### Import libraries

In [67]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow import keras
import shutil
import matplotlib.image as mpimg
import glob
import os

## Prepare the data

In [68]:
# Model / data parameters
img_width, img_height = 28, 28
epochs = 50
batch_size = 16
input_shape = (28, 28, 1)
num_classes = 10

notMNIST_load_data is a function that loads the notMNIST dataset. It returns three tuples containing the training, validation and test data. Each tuple is formed by a numpy array containing the images and a numpy array containing the labels (0 to 9).

In [69]:
def notMNIST_load_data() : 
    data_dir_letters = 'data/notMNIST_small'

    nb_letters = len(os.listdir(data_dir_letters))
    if nb_letters != num_classes:
        raise ValueError('The number of classes is not equal to the number of letters in the folder')

    x_train = []
    y_train = []
    x_test = []
    y_test = []

    # we get the number of samples in the smallest class
    min_nb_samples = float('inf')
    for letter in os.listdir(data_dir_letters):
        nb_samples = len(os.listdir(os.path.join(data_dir_letters, letter)))
        min_nb_samples = min(min_nb_samples, nb_samples)

    # 80% of the data is used for training
    # 10% for validation
    # 10% for testing
    nb_train_samples = int(min_nb_samples * 0.8)
    # nb_validation_samples = int(min_nb_samples * 0.1)
    nb_validation_samples = 0
    nb_test_samples = nb_samples - nb_train_samples - nb_validation_samples

    # for each letter folder, we copy the images in the train, validation or test tuple and the label
    # TODO : randomize the order of the images would be better ?
    for letter in os.listdir(data_dir_letters):
        index = 0

        for image in glob.iglob(os.path.join(data_dir_letters, letter, "*.png")):

            if index < nb_train_samples:
                pixels_array = mpimg.imread(image)
                x_train.append(pixels_array)
                y_train.append(ord(letter) - 65)
            #elif index < nb_train_samples + nb_validation_samples:
                
            elif index < nb_train_samples + nb_validation_samples + nb_test_samples:
                pixels_array = mpimg.imread(image)
                x_test.append(pixels_array)
                y_test.append(ord(letter) - 65)
            index += 1
    
    return (np.array(x_train), np.array(y_train)), (np.array(x_test), np.array(y_test))

In [70]:
# load the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = notMNIST_load_data()

In [71]:
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [72]:
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape: (14970, 28, 28, 1)
14970 train samples
3750 test samples


In [73]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Build the model

In [77]:
def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dropout(rate=0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [78]:
# get the model
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_5 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 32)        9248      
                                                                 
 activation_6 (Activation)   (None, 11, 11, 32)        0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 5, 5, 32)          0         
 g2D)                                                 

In [79]:
# compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy']
)

## Train the model

In [80]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15


106/106 [==============================] - 4s 20ms/step - loss: 2.2479 - accuracy: 0.1098 - val_loss: 4.1329 - val_accuracy: 0.0000e+00
Epoch 2/15
106/106 [==============================] - 2s 18ms/step - loss: 2.2226 - accuracy: 0.1118 - val_loss: 4.3945 - val_accuracy: 0.0000e+00
Epoch 3/15
106/106 [==============================] - 2s 16ms/step - loss: 2.1754 - accuracy: 0.1549 - val_loss: 5.7734 - val_accuracy: 0.0000e+00
Epoch 4/15
106/106 [==============================] - 2s 15ms/step - loss: 1.8159 - accuracy: 0.3223 - val_loss: 9.5265 - val_accuracy: 0.0000e+00
Epoch 5/15
106/106 [==============================] - 2s 15ms/step - loss: 1.5230 - accuracy: 0.4269 - val_loss: 11.4054 - val_accuracy: 0.0000e+00
Epoch 6/15
106/106 [==============================] - 2s 16ms/step - loss: 1.3900 - accuracy: 0.4925 - val_loss: 12.2368 - val_accuracy: 0.0000e+00
Epoch 7/15
106/106 [==============================] - 2s 15ms/step - loss: 1.2856 - accuracy: 0.5416 - val_loss: 1

## Evaluate the trained model

In [81]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 2.3677165508270264
Test accuracy: 0.6389333605766296
